In [1]:
import numpy as np
import os
import torch
import torchvision
import cv2
from torchvision import models
from torchvision import transforms
from  PIL import Image


In [2]:
# my models in pytorch
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vg

In [3]:
# do some reaseach on this-
resnet = models.resnet101(pretrained=True)

In [4]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )])

In [6]:
path = r'C:\Users\richa\Desktop\projects\Bristol uni porjects\Univecity projects\Applied Data Science\uob_image_set'

In [7]:
def load_images_from_folder(folder):
        
    images = []
    files = folders = 0
    count = 0
    dirnames = [] 

    for _, dirnames, filenames in os.walk(folder):

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])  
        for filename in os.listdir(current_path):
            img = cv2.imread(os.path.join(current_path, filename))
            if img is not None:
                
                images.append(img)
#                 img = torch.from_numpy(img).type(torch.uint8)
#                 images.append(img)

        count += 1
        
        if count % 10 == 0:
            print(count)
        
        # only do 30 out of the 1500 folders...is too big otherwise
        if count == 30:
            return images
    return images


In [8]:
def load_images_from_folder(folder, end, as_tensor = False):

    images = []
    files = folders = 0
    count= 0
    dirnames = [] 

 

    for _, dirnames, filenames in os.walk(folder):

 

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])
        for filename in os.listdir(current_path):
            img = cv2.imread(os.path.join(current_path, filename))
            if img is not None:
                if as_tensor:
                    img = torch.from_numpy(img * 1.0)
                    # img = torch.from_numpy(img).float()
                    images.append(img)
                else: 
                    img = Image.fromarray(img)
#                     print(type(img))
                    images.append(img)

 

        count += 1
        
        if count % 10 == 0:
            print('number of folder done =', count, 'total number of images so far =', len(images))
            
        if count  == end:
            return images
        # only do 300 out of the 1500 folders...is too big otherwise
        # if count == 100:
        #     return images
    return images

In [9]:
dataset = load_images_from_folder(path, 30)

number of folder done = 10 total number of images so far = 42
number of folder done = 20 total number of images so far = 87
number of folder done = 30 total number of images so far = 128


In [10]:
dataset

[<PIL.Image.Image image mode=RGB size=1000x1334 at 0x1207AF32E20>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x1207AF32910>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x1207AF324F0>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014CD0>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014580>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014970>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x120090149A0>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014A30>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014280>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014D60>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014820>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014D90>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014A90>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x12009014640>,
 <PIL.Image.Image image mode=RGB size=1000x1334 at 0x120090149

In [11]:
list_input_tensor = []
list_input_batch = []
num_img = 15
for item in range(num_img):
    input_tensor = preprocess(dataset[item])
    input_batch = input_tensor.unsqueeze(0)
    list_input_tensor.append(input_tensor)
    list_input_batch.append(input_batch)
    

In [12]:
# input_tensor_1 = preprocess(dataset[23])
# input_batch_1 = input_tensor.unsqueeze(0)
# input_tensor_2 = preprocess(dataset[23])
# input_batch_2 = input_tensor.unsqueeze(0)
# input_tensor_3 = preprocess(dataset[23])
# input_batch_3 = input_tensor.unsqueeze(0)

In [13]:
torch.cuda.is_available()

True

In [14]:
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     resnet.to('cuda')

# with torch.no_grad():
#     output = resnet(input_batch)
# # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(max(probabilities))

In [15]:
for param in resnet.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 3)







# model_conv = resnet.to('cuda')

# criterion = torch.nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = torch.optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [16]:

for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
    print(output)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(max(probabilities))

tensor([[ 0.2483,  0.0845, -0.0477]], device='cuda:0')
tensor([[ 0.2204,  0.1013, -0.0543]], device='cuda:0')
tensor([[ 0.1953,  0.0844, -0.0062]], device='cuda:0')
tensor([[ 0.2266,  0.0698, -0.0538]], device='cuda:0')
tensor([[ 0.2149,  0.1264, -0.0377]], device='cuda:0')
tensor([[ 0.2218,  0.1473, -0.0255]], device='cuda:0')
tensor([[ 0.2065,  0.0982, -0.0364]], device='cuda:0')
tensor([[ 0.2203,  0.1481, -0.0091]], device='cuda:0')
tensor([[ 0.2402,  0.1075, -0.0333]], device='cuda:0')
tensor([[ 0.2397,  0.1256, -0.0015]], device='cuda:0')
tensor([[ 0.2253,  0.1127, -0.0158]], device='cuda:0')
tensor([[ 0.2557,  0.1187, -0.0279]], device='cuda:0')
tensor([[ 0.1986,  0.1203, -0.0720]], device='cuda:0')
tensor([[ 0.2076,  0.1095, -0.0564]], device='cuda:0')
tensor([[ 0.1962,  0.1059, -0.0557]], device='cuda:0')


In [19]:
output_array = []
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
        cup_tensor = output.cpu()
        
    output_array.append(cup_tensor[0].tolist())
#     output_array.append(np.asarray(cup_tensor[0]))
#     print(np.asarray(cup_tensor[0]))
output_array = np.asarray(output_array)
display(output_array)

array([[ 0.24833271,  0.08452591, -0.04774975],
       [ 0.2203742 ,  0.1012618 , -0.05426013],
       [ 0.19531453,  0.08436403, -0.00623201],
       [ 0.22655743,  0.06982467, -0.05378487],
       [ 0.21489754,  0.12641831, -0.03771731],
       [ 0.22178392,  0.14733088, -0.02548935],
       [ 0.20649591,  0.09823374, -0.03635781],
       [ 0.22032006,  0.14813614, -0.00907289],
       [ 0.24017271,  0.10754865, -0.03326272],
       [ 0.23967849,  0.12562047, -0.00152951],
       [ 0.22534187,  0.11270872, -0.01581158],
       [ 0.25572148,  0.11873488, -0.02789681],
       [ 0.19859871,  0.12034151, -0.07197057],
       [ 0.20763674,  0.10948252, -0.05638269],
       [ 0.19617376,  0.10589571, -0.05566626]])

In [20]:
output_array[:,0]

array([0.24833271, 0.2203742 , 0.19531453, 0.22655743, 0.21489754,
       0.22178392, 0.20649591, 0.22032006, 0.24017271, 0.23967849,
       0.22534187, 0.25572148, 0.19859871, 0.20763674, 0.19617376])

In [21]:
import json
import matplotlib.pyplot as plt
import itertools
import random
from scipy.interpolate import *
from mpl_toolkits.mplot3d import axes3d

In [24]:
import pylab
import numpy 
import ipympl

# %matplotlib widget
# %matplotlib notebook

# x = [item[0] for item in output_array]
# y = [item[1] for item in output_array]
# z = [item[2] for item in output_array]
# # output_array[:,0], output_array[:,1], output_array[:,2]

# output_array_1 = numpy.array(output_array)
# print(output_array_1)

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter(output_array[:,0], output_array[:,1], output_array[:,2],color='r',s=3)
ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

# length = data.shape[0]
# width = data.shape[1]
# x, y = np.meshgrid(np.arange(length), np.arange(width))

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1, projection='3d')
# ax.plot_surface(x, y, data)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …